In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv("Pipes_Break20.csv")

print(df.columns)
# # select non join columns and ML needed columns
columns = ['OBJECTID', 'DWW_Mainlines__Permitted_Use__MNL_FEAT_1', 'DWW_Mainlines__Permitted_Use__MNL_MATE_1', 
       'DWW_Mainlines__Permitted_Use__MNL_LENGTH', 'DWW_Mainlines__Permitted_Use__MNL_INSTAL',
       'Pipe_widths_Width', 'NEAR_DIST', 'MUSYM', 'ARTCLASS',
       'SPEEDLIMIT', 'SURFACEWID', 'SURFACETYP', 'SLOPE_PCT', 'Size', 'DATE']
non_joins = df[columns]
non_joins.head(10)

Index(['OBJECTID', 'Join_Count', 'Join_Count_1',
       'FID_DWW_Mainlines__Permitted_Use__Pipe_widths',
       'DWW_Mainlines__Permitted_Use__MNL_FEA_KE',
       'DWW_Mainlines__Permitted_Use__MNL_FEAT_1',
       'DWW_Mainlines__Permitted_Use__MNL_OWNE_1',
       'DWW_Mainlines__Permitted_Use__MNL_MATE_1',
       'DWW_Mainlines__Permitted_Use__MNL_LENGTH',
       'DWW_Mainlines__Permitted_Use__MNL_INSTAL', 'Pipe_widths_MNL_FEA_KEY',
       'Pipe_widths_Width', 'FID_SeaSoilClip', 'MUSYM', 'MUKEY', 'muname',
       'ARTCLASS', 'UNITDESC', 'STNAME_ORD', 'BLOCKNBR', 'SPEEDLIMIT',
       'SURFACEWID', 'SURFACETYP', 'SLOPE_PCT', 'NEAR_DIST', 'NEAR_X',
       'NEAR_Y', 'WONUM', 'ASSETNUM', 'Size', 'Long', 'Lat', 'DATE'],
      dtype='object')


,OBJECTID,DWW_Mainlines__Permitted_Use__MNL_FEAT_1,DWW_Mainlines__Permitted_Use__MNL_MATE_1,DWW_Mainlines__Permitted_Use__MNL_LENGTH,DWW_Mainlines__Permitted_Use__MNL_INSTAL,Pipe_widths_Width,NEAR_DIST,MUSYM,ARTCLASS,SPEEDLIMIT,SURFACEWID,SURFACETYP,SLOPE_PCT,Size,DATE
0,1,Mainline,Concrete,314.79,1/1/1972 0:00:00,8.0,3964.600720,3055,2.0,25.0,40.0,PCC,0.0,NaN,NaN
1,2,Mainline,Concrete,363.39,1/1/1972 0:00:00,8.0,3609.210948,3056,0.0,20.0,46.0,AC,4.0,NaN,NaN
2,3,Mainline,Concrete,323.51,1/1/1972 0:00:00,8.0,3451.254435,3056,0.0,20.0,46.0,AC,4.0,NaN,NaN
3,4,Mainline,Vitrified Clay,329.13,1/1/1928 0:00:00,12.0,833.915482,3056,0.0,20.0,0.0,ST,6.0,NaN,NaN
4,5,Mainline,Reinforced Concrete Pipe,273.64,1/1/1928 0:00:00,18.0,852.426502,3056,2.0,25.0,42.0,AC/PCC,4.0,NaN,NaN
5,6,Mainline,Concrete,30.87,1/1/1963 0:00:00,8.0,1404.924372,3056,0.0,20.0,23.0,AC,3.0,NaN,NaN
6,7,Mainline,Reinforced Concrete Pipe,112.29,1/1/1990 0:00:00,18.0,2212.976394,988,0.0,20.0,22.0,ST,2.0,NaN,NaN
7,8,Stub,Concrete,29.89,1/1/1958 0:00:00,8.0,3065.662515,3055,0.0,20.0,21.0,ST,1.0,NaN,NaN
8,9,Stub,Concrete,7.49,1/1/1958 0:00:00,8.0,3148.924667,3055,0.0,20.0,21.0,ST,2.0,NaN,NaN
9,10,Mainline,Concrete,42.59,1/1/1958 0:00:00,8.0,3196.074038,3059,0.0,20.0,18.0,ST,11.0,NaN,NaN


In [4]:
ddff = non_joins[non_joins['DATE'].notna()]
print(len(ddff['DATE']))
print(len(ddff['DATE'].unique()))
print('duplicate breaks: ', len(ddff['DATE']) - len(ddff['DATE'].unique()))

2800
1951
duplicate breaks:  849


In [5]:
non_joins = non_joins[(~non_joins['DATE'].duplicated() | df['DATE'].isnull())]

In [6]:
# select only years
non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'] = pd.to_datetime(non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'], format='%m/%d/%Y %H:%M:%S')
non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'] = non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'].map(lambda x: x.year)

non_joins['DATE'] = pd.to_datetime(non_joins['DATE'], format='%m/%d/%Y %H:%M:%S')
non_joins['DATE'] = non_joins['DATE'].map(lambda x: x.year)

non_joins.head()

,OBJECTID,DWW_Mainlines__Permitted_Use__MNL_FEAT_1,DWW_Mainlines__Permitted_Use__MNL_MATE_1,DWW_Mainlines__Permitted_Use__MNL_LENGTH,DWW_Mainlines__Permitted_Use__MNL_INSTAL,Pipe_widths_Width,NEAR_DIST,MUSYM,ARTCLASS,SPEEDLIMIT,SURFACEWID,SURFACETYP,SLOPE_PCT,Size,DATE
0,1,Mainline,Concrete,314.79,1972,8.0,3964.600720,3055,2.0,25.0,40.0,PCC,0.0,NaN,NaN
1,2,Mainline,Concrete,363.39,1972,8.0,3609.210948,3056,0.0,20.0,46.0,AC,4.0,NaN,NaN
2,3,Mainline,Concrete,323.51,1972,8.0,3451.254435,3056,0.0,20.0,46.0,AC,4.0,NaN,NaN
3,4,Mainline,Vitrified Clay,329.13,1928,12.0,833.915482,3056,0.0,20.0,0.0,ST,6.0,NaN,NaN
4,5,Mainline,Reinforced Concrete Pipe,273.64,1928,18.0,852.426502,3056,2.0,25.0,42.0,AC/PCC,4.0,NaN,NaN


In [8]:
# filter out NaN from DATE col
breaks_df = non_joins[non_joins['DATE'].notna()]
# set size column to width column and drop size 
breaks_df['Pipe_widths_Width'] = breaks_df['Size']
breaks_df = breaks_df.drop('Size', axis=1)
breaks_df.head()

,OBJECTID,DWW_Mainlines__Permitted_Use__MNL_FEAT_1,DWW_Mainlines__Permitted_Use__MNL_MATE_1,DWW_Mainlines__Permitted_Use__MNL_LENGTH,DWW_Mainlines__Permitted_Use__MNL_INSTAL,Pipe_widths_Width,NEAR_DIST,MUSYM,ARTCLASS,SPEEDLIMIT,SURFACEWID,SURFACETYP,SLOPE_PCT,DATE
47,48,Mainline,Concrete,40.54,1957,6,699.766911,3056,0.0,20.0,20.0,AC,2.0,2014.0
100,101,Mainline,Reinforced Concrete Pipe,25.65,1973,8,2296.768732,3055,0.0,20.0,38.0,ST,4.0,2017.0
101,102,Mainline,Reinforced Concrete Pipe,25.65,1973,8,2296.768732,3055,0.0,20.0,38.0,ST,4.0,2017.0
338,339,Mainline,Vitrified Clay,73.96,1929,12,1363.416655,3056,0.0,20.0,24.0,PCC,2.0,2013.0
374,375,Mainline,Reinforced Concrete Pipe,769.29,2005,8,0.000000,988,1.0,25.0,52.0,AC,0.0,2011.0


In [16]:
pseudo_df = non_joins
# pseudo_df.head()

# fill Nan with 0 (idk why, but it just made it work FOR NOW)
# if date is not 0 (NaN), make width = size 
# then for all df, drop size column

pseudo_df['DATE'] = pseudo_df['DATE'].fillna(0)
pseudo_df.loc[pseudo_df['DATE'] != 0, ['Pipe_widths_Width']] = pseudo_df['Size']
pseudo_df = pseudo_df.drop('Size', axis=1)
# pseudo_df.head()

# change 'Width' values to numbers instead of strings (for dummy prep)
pseudo_df['Width'] = pd.to_numeric(pseudo_df['Pipe_widths_Width'], errors='coerce')
pseudo_df = pseudo_df.drop('Pipe_widths_Width', axis = 1)
# pseudo_df['Width'].unique()

# Assign binary variables:
# [Create new column] If pipe has a broken date --> broken pipes = 0, non-broken pipes = 1
pseudo_df['TARGET'] = pseudo_df['DATE'].apply(lambda x: 0 if x == 0 else 1)

In [17]:
# Create dummy variables
dummy_df = pd.get_dummies(pseudo_df)
dummy_df.head()

,OBJECTID,DWW_Mainlines__Permitted_Use__MNL_LENGTH,DWW_Mainlines__Permitted_Use__MNL_INSTAL,NEAR_DIST,MUSYM,ARTCLASS,SPEEDLIMIT,SURFACEWID,SLOPE_PCT,DATE,...,DWW_Mainlines__Permitted_Use__MNL_MATE_1_Unknown,DWW_Mainlines__Permitted_Use__MNL_MATE_1_Vitrified Clay,DWW_Mainlines__Permitted_Use__MNL_MATE_1_Wood Stave Pipe,SURFACETYP_,SURFACETYP_AC,SURFACETYP_AC/AC,SURFACETYP_AC/PCC,SURFACETYP_GRAVEL,SURFACETYP_PCC,SURFACETYP_ST
0,1,314.79,1972,3964.600720,3055,2.0,25.0,40.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,2,363.39,1972,3609.210948,3056,0.0,20.0,46.0,4.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,3,323.51,1972,3451.254435,3056,0.0,20.0,46.0,4.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,4,329.13,1928,833.915482,3056,0.0,20.0,0.0,6.0,0.0,...,0,1,0,0,0,0,0,0,0,1
4,5,273.64,1928,852.426502,3056,2.0,25.0,42.0,4.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
def get_data(df, start, end):
    """
    Takes in df and filters depending on timeframe (start and end years).
    Returns subset of data for training in timeframe.
    """
    # want to include where there is NOT a break year (those will be our non-broken positive examples --> 'DATE' == 0) - DATE col
    # want to include where break year is in time frame of what we want - DATE col
    train_df = df[(df['DATE'] == 0 )| ((df['DATE'] >= start) & (df['DATE'] <= end))]

    # exclude installs AFTER time frame window - MNL_INSTAL col
    train_df = train_df[(train_df['DWW_Mainlines__Permitted_Use__MNL_INSTAL'] <= end)]

    # based on time window, calculate appropriate age of pipes (select beginning year of time frame --> ex: 2009, 2010, 2011
    #       subtract install year from 2009) - MNL_INSTAL col
    # -- will create negative numbers
    train_df['AGE'] = start - train_df['DWW_Mainlines__Permitted_Use__MNL_INSTAL']

    return train_df

In [12]:
train_df = get_data(dummy_df, 2009, 2011)
# train_df.index

In [13]:
# trains
# tests
# move over
# rinse & repeat until 2019
def split_df(df):
    """
    """    
    df = df.dropna()
    feature_df = df.drop(['TARGET', 'DATE'], axis=1)
    target_df = df[['TARGET']]

    return feature_df, target_df

def train_v1(df, model):
    """
    Takes in a dataframe of interest and a model, and trains the model.
    Model likely needs to be one imported from sklearn so it is able to
    handle dataframes as input data
    """
    feature, target = split_df(df)
    clf = model #clf is for for classification
    clf.fit(feature, target)

    return clf

In [14]:
def main_function(dummy_df, start, end, model):
    """
    Takes in dummy dataframe and runs all functions based on given year ranges.
    Prints out year and accuracy per time range (3 years ranges)
    """

    full_results = {} # Dictionary for storing all the models and their data

    for i in range(start, end - 4):

        

        start_train = i
        end_train = i + 2
        df = get_data(dummy_df, start_train, end_train)
        feature_train, target_train = split_df(df)
        clf = train_v1(df, model)

        training_pred = clf.predict(feature_train)
        #print(start_train, "-", end_train, ": ")#, accuracy_score(y_true=target, y_pred=training_pred)) 

        # Test
        start_test = i + 3
        end_test = start_test + 2
        test = get_data(dummy_df, start_test, end_test)
        feature_test, target_test = split_df(test)
        testing_pred = clf.predict(feature_test)
        #testing_proba_preds = clf.predict_proba(feature_test)[::,1]
        #print(start_test, "-", end_test, ": ", accuracy_score(y_true=target_test, y_pred=testing_pred))
        
        print(start_train, "-", end_test, ": ")#, accuracy_score(y_true=target, y_pred=training_pred)) 


        # Print out break and non break accuracy's rather total full accuracy
        cm = confusion_matrix(y_true=target_test, y_pred=testing_pred)
        # nonbreak_acc = cm[0,0] / (cm[0,0] + cm[0,1])
        # print('Non-break accuracy: ', nonbreak_acc)
        # break_acc = cm[1,1] / (cm[1, 0] + cm[1,1])
        # print('Break accuracy: ', break_acc)

        #print out metrics
        # precision, recall, f1, support = precision_recall_fscore_support(y_true=target_test, y_pred=testing_pred)
        # print('Precision: ', precision)
        # print('Recall: ', recall)
        # print('F1 Score: ', f1)
        # print('Support: ', support)

        precision = precision_score(y_true=target_test, y_pred=testing_pred)
        recall = recall_score(y_true=target_test, y_pred=testing_pred)
        bal_acc = balanced_accuracy_score(y_true=target_test, y_pred=testing_pred)
        print('precision: ', precision)
        print('recall: ', recall)
        print('Balanced accuracy: ', bal_acc)

        print()

        model_results = {
            'model': clf,
            'start_year': start_test,
            'end_year': end_test,
            'test_df': feature_test, # For eventual SHAP calculations
            'target_values': target_test,
            'test_predictions': testing_pred,
            'confusion_matrix': cm

            }

        full_results['model_' + str(start_test) + '-' + str(end_test)] = model_results
    
    return full_results

In [15]:
gbt = GradientBoostingClassifier() # ~58% accuracy
ada = AdaBoostClassifier() # 45-52% accuracy
#stc = StackingClassifier()
rf = RandomForestClassifier()
bag = BaggingClassifier()
etc = ExtraTreesClassifier()#max_depth=100, max_features='log2', n_estimators=100) # ~58% accuracy
lr = LogisticRegression()

In [16]:
param_grid = { 'C': [1.0, 1e3, 1e6],
    'solver' : ['lbfgs','newton-cg', 'sag', 'saga'],
    'max_iter' : [100, 150, 200, 250, 300, 350, 400, 450, 500],
    #'class_weight' : ['balanced', 'None']
   
 }


cv_model = RandomizedSearchCV(estimator=lr, param_distributions=param_grid, scoring='recall', n_iter=10)

X_df, y_df = split_df(train_df)
cv_model.fit(X_df, y_df)

KeyboardInterrupt: 

In [15]:
results = main_function(dummy_df, 2009, 2019, cv_model.best_estimator_)
print(results.keys())

2009 - 2014 : 
precision:  0.9926739926739927
recall:  0.5103578154425612
Balanced accuracy:  0.7551613721679462

2010 - 2015 : 
precision:  1.0
recall:  0.4703891708967851
Balanced accuracy:  0.7351945854483926



KeyboardInterrupt: 

In [16]:
cv_model.best_params_

{'solver': 'newton-cg', 'max_iter': 500, 'C': 1000000.0}

In [17]:
cv_model.best_score_

0.5961616161616161